# RAG with Azure Cosmos DB for Mongo vcore
You can run this notebook after running succesfully the "RAG - Cosmos for Mongo - create embeddings" notebook. 

In [ ]:
from dotenv import load_dotenv
import pandas as pd
from IPython.display import display, HTML, JSON, Markdown
import os

# Configure environment variables
load_dotenv()

# Configure OpenAI API
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME  = os.getenv("AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")


#configure Cosmos 
COSMOS_MONGO_CONNECTION_STRING = os.getenv("COSMOS_MONGO_CONNECTION_STRING")
COSMOS_INDEX_NAME = os.getenv("COSMOS_INDEX_NAME")
COSMOS_DBNAME = os.getenv("COSMOS_DBNAME")
COSMOS_COLLECTION_NAME = os.getenv("COSMOS_COLLECTION_NAME")

In [7]:
from langchain_openai import AzureOpenAIEmbeddings
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [12]:
embeddingmodel = AzureOpenAIEmbeddings(
    deployment=SWEDEN_AZURE_OPENAI_ENDPOINT,
    model=SWEDEN_OPENAI_EMBEDDNG_ADA2_DEPLOYMENT_NAME,
    azure_endpoint=SWEDEN_OPENAI_EMBEDDNG_ADA2_DEPLOYMENT_NAME,
    api_key=SWEDEN_AZURE_OPENAI_API_KEY,
    chunk_size = 1)

In [13]:
#we use the tenacity library to create delays and retries when calling openAI embeddings to avoid hitting throttling limits
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def calc_embeddings(text):
    deployment = SWEDEN_OPENAI_EMBEDDNG_ADA2_DEPLOYMENT_NAME
    # replace newlines, which can negatively affect performance.
    txt = text.replace("\n", " ")
    return embeddingmodel.embed_query(txt)

In [15]:
from pymongo import MongoClient
from langchain_community.vectorstores.azure_cosmos_db import (
    AzureCosmosDBVectorSearch,
    CosmosDBSimilarityType,
)

client: MongoClient = MongoClient(COSMOS_MONGO_CONNECTION_STRING)

# Create database if it doesn't exist
db = client[COSMOS_DBNAME]
if COSMOS_DBNAME not in client.list_database_names():
    # Create a database with 400 RU throughput that can be shared across
    # the DB's collections
    db = client[COSMOS_DBNAME]
    print("Created db '{}'.\n".format(COSMOS_DBNAME))
else:
    print("Using database: '{}'.\n".format(COSMOS_DBNAME))

# Create collection if it doesn't exist
collection = db[COSMOS_COLLECTION_NAME]
if COSMOS_COLLECTION_NAME not in db.list_collection_names():
    # Creates a unsharded collection that uses the DBs shared throughput
    collection = db[COSMOS_COLLECTION_NAME]
    print("Created collection '{}'.\n".format(COSMOS_COLLECTION_NAME))
else:
    print("Using collection: '{}'.\n".format(COSMOS_COLLECTION_NAME))

collection = client[COSMOS_DBNAME][COSMOS_COLLECTION_NAME]

C:\Users\dschlesinger\AppData\Local\Temp\ipykernel_2904\2728352432.py:7: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client: MongoClient = MongoClient(COSMOS_MONGO_CONNECTION_STRING)


Using database: 'embed-test-db'.

Using collection: 'embed-test-collection'.



In [17]:
vectorstore = AzureCosmosDBVectorSearch(
    collection, embeddingmodel, index_name=COSMOS_INDEX_NAME
)

# perform a similarity search between a query and the ingested documents
question = "Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?"
docs = vectorstore.similarity_search(question)
answer = docs[0].page_content
print(answer)


APIConnectionError: Connection error.

In [7]:
# use GPT to answer a question based on the question and the answer from our similarity search
from openai import AzureOpenAI
clientOpenAI = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)

def call_openAI(text):
    response = clientOpenAI.chat.completions.create(
        model=OPENAI_GPT35_DEPLOYMENT_NAME,
        messages = text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response.choices[0].message.content

In [8]:
prompt = 'Question: {}'.format(question) + '\n' + 'Information: {}'.format(answer)
# prepare prompt
messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users questions. Answer the question using the provided information and do not add anything else."},
            {"role": "user", "content": prompt}]

result = call_openAI(messages)
display(HTML(result))